In [30]:
import sys
import spotipy
import spotipy.util as util
import pandas as pd
import numpy as np
sp = spotipy.Spotify()

def get_playlist_tracks(credentials,username,playlist_id):
    #set scope to retreive public data
    scope_playlist = 'playlist-modify-public'
    #provide token using given credentials
    #credentials are: username, public key, private key and redirect_uri
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    #get tracklisting from playlist data
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    #ensure collection of data after more than 100 requests
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

def get_features(tracks):
    #set scope to retreive public data
    scope_playlist = 'playlist-modify-public'
    #provide token using given credentials
    #credentials are: username, public key, private key and redirect_uri
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    #make empty list to collect dictionaries for dataframe
    dataframe = []
    for x in tracks:
        try:
            track = x['track']
            #get audio features per track
            feat = (sp.audio_features(tracks=[track['id']])[0])
            #get track and artist info
            art_dict = {'artist':track['artists'][0]['name'], 'title':track['name'], 'popularity':track['popularity']}
            #combine both dictionaries
            art_dict.update(feat)
            #make dataframe from dictionary
            dataframe.append(art_dict)
        except:
            print("Oh no!",sys.exc_info()[0],"happend")
    #filter dataframe to neccesary columns
    df = pd.DataFrame(dataframe)
    df = df.filter(['title', 'artist', 'danceability','energy', 'loudness', 'tempo','valence', 'popularity','id'])
    df = df.set_index('title')
    return df

def add_tracks_from_df(credentials,playlist,dataframe,features):
    scope_playlist = 'playlist-modify-public'
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    #make selection based on feature input
    selection = dataframe.loc[(dataframe['danceability'].between(features[0]-0.1, features[0]+0.1, inclusive=False)) &\
    (dataframe['energy'].between(features[1]-0.2, features[1]+0.2, inclusive=False)) &\
    (dataframe['loudness'].between(features[2]-5, features[2]+5, inclusive=False)) &\
    (dataframe['tempo'].between(features[3]-40, features[3]+40, inclusive=False)) &\
    (dataframe['valence'].between(features[4]-0.5, features[4]+0.5, inclusive=False)) &\
    #remove unwanted artists
    (~dataframe['artist'].isin(['Ariana Grande']))]
    selection.sort_values(by=['popularity'], ascending=False)
    songs = selection['id'].tolist()
    
    scope_playlist = 'playlist-modify-public'
    token2 = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp2 = spotipy.Spotify(auth=token2)
    sp2.user_playlist_add_tracks(credentials[0],playlist, songs)
    print("Songs Succesfully Added to Playlist")
    
def remove_tracks_from_df(credentials,playlist,dataframe,features):
    scope_playlist = 'playlist-modify-public'
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    selection = dataframe.loc[(dataframe['danceability'].between(features[0]-0.1, features[0]+0.1, inclusive=False)) &\
    (dataframe['energy'].between(features[1]-0.2, features[1]+0.2, inclusive=False)) &\
    (dataframe['loudness'].between(features[2]-5, features[2]+5, inclusive=False)) &\
    (dataframe['tempo'].between(features[3]-40, features[3]+40, inclusive=False)) &\
    (dataframe['valence'].between(features[4]-0.5, features[4]+0.5, inclusive=False))]
    selection.sort_values(by=['popularity'], ascending=False)
    songs = selection['id'].tolist()
    
    scope_playlist = 'playlist-modify-public'
    token2 = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp2 = spotipy.Spotify(auth=token2)
    sp2.user_playlist_remove_all_occurrences_of_tracks(credentials[0],playlist, songs)
    print("Songs Succesfully Removed from Playlist")


In [33]:
credentials = ['zwamborn', '5711bc132b4c48ceb5bbd19cd65b1e63', 'f507991961c948d8bf1b62ae6ef5ab15', 'http://localhost']
tracklist = get_playlist_tracks(credentials, 'tillyxh','1TFZPafSKEeh31U9fky1vW')

In [35]:
data = get_features(tracklist)

In [37]:
features = [0.7,0.5,-5,170,0.4]
add_tracks_from_df(credentials,'4jwcDPU0HyEw0cOAPPFOdp',data,features)

Songs Succesfully Added to Playlist


In [38]:
remove_tracks_from_df(credentials,'4jwcDPU0HyEw0cOAPPFOdp',data,features)

Songs Succesfully Removed from Playlist


In [39]:
data

,artist,danceability,energy,loudness,tempo,valence,popularity,id
title,,,,,,,,
Intro,The xx,0.617,0.778,-8.871,100.363,0.152,71,0bXpmJyHHYPk6QBFj25bYF
NY Excuse,Soulwax,0.666,0.731,-5.902,133.347,0.790,38,7rnAXmOwAQkn2nZaae1Xfs
Somebody Better,Black Honey,0.331,0.881,-2.637,135.996,0.219,25,18WXsmmjUGlvqAccJUbCLF
CASTLE OF GLASS,Linkin Park,0.615,0.911,-3.584,108.000,0.685,70,1r1fPuhj9H4VdXr7OK6FL5
Arabella,Arctic Monkeys,0.579,0.558,-6.986,179.949,0.506,72,7nzsY8vlnKdvGOEE0rjAXZ
Black Treacle,Arctic Monkeys,0.343,0.890,-5.759,100.061,0.525,56,7EUcOIbJSePmXGvAe7G79C
Stop The World I Wanna Get Off With You,Arctic Monkeys,0.661,0.730,-6.779,95.027,0.530,60,2QUlMbvD9bL8wHqfsoqoet
Sweet Nothing (feat. Florence Welch),Calvin Harris,0.573,0.929,-3.942,127.934,0.582,70,24LS4lQShWyixJ0ZrJXfJ5
What The Water Gave Me,Florence + The Machine,0.483,0.572,-6.410,124.112,0.187,53,10H6mvk1cPPTzs3IHsHSHi
